In [38]:
import sys

sys.path.append('../')
from auton_survival import datasets
outcomes, features = datasets.load_support()

In [39]:
from auton_survival.preprocessing import Preprocessor

cat_feats = ['sex', 'dzgroup', 'dzclass', 'income', 'race', 'ca']
num_feats = ['age', 'num.co', 'meanbp', 'wblc', 'hrt', 'resp',
             'temp', 'pafi', 'alb', 'bili', 'crea', 'sod', 'ph',
             'glucose', 'bun', 'urine', 'adlp', 'adls']


features = Preprocessor().fit_transform(features, cat_feats=cat_feats, num_feats=num_feats)

In [40]:
import numpy as np

horizons = [0.25, 0.5, 0.75]
times = np.quantile(outcomes.time[outcomes.event==1], horizons).tolist()

In [41]:
x, t, e = features.values, outcomes.time.values, outcomes.event.values

n = len(x)

tr_size = int(n * 0.70)
vl_size = int(n * 0.10)
te_size = int(n * 0.20)

x_train, x_test, x_val = x[:tr_size], x[-te_size:], x[tr_size:tr_size+vl_size]
t_train, t_test, t_val = t[:tr_size], t[-te_size:], t[tr_size:tr_size+vl_size]
e_train, e_test, e_val = e[:tr_size], e[-te_size:], e[tr_size:tr_size+vl_size]

t = outcomes["time"]
e = outcomes["event"]
quantiles = [0.25, 0.5, 0.75]
quantiles = np.quantile(t[e == 1], quantiles)
quantiles

array([ 14.,  58., 252.])

In [42]:
from sklearn.model_selection import ParameterGrid

In [43]:
param_grid = {'k' : [3, 4, 6],
              'learning_rate' : [1e-4, 1e-3],
              'layers' : [ [], [100], [100, 100] ]
             }
params = ParameterGrid(param_grid)

In [48]:
from auton_survival.models.dcm import DeepCoxMixtures

model = DeepCoxMixtures()

# The fit method is called to train the model
model.fit(x_train, t_train, e_train, iters=100, val_data=(x_val, t_val, e_val))

  0%|          | 0/100 [00:00<?, ?it/s]D:\anaconda3\envs\FLCausalMixture\lib\site-packages\auton_survival\models\dcm\dcm_utilities.py:105: RuntimeWarning: invalid value encountered in log
  probs = gates+np.log(event_probs)
  1%|          | 1/100 [00:00<00:49,  1.99it/s]D:\anaconda3\envs\FLCausalMixture\lib\site-packages\auton_survival\models\dcm\dcm_utilities.py:105: RuntimeWarning: divide by zero encountered in log
  probs = gates+np.log(event_probs)
 90%|█████████ | 90/100 [01:06<00:07,  1.35it/s]


In [52]:
out_risk = model.predict_survival(x_test, times)
out_survival = model.predict_survival(x_test, times)

In [53]:
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

In [54]:
cis = []
brs = []

et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                 dtype = [('e', bool), ('t', float)])
et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                 dtype = [('e', bool), ('t', float)])
et_val = np.array([(e_val[i], t_val[i]) for i in range(len(e_val))],
                 dtype = [('e', bool), ('t', float)])

for i, _ in enumerate(times):
    cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:, i], times[i])[0])
brs.append(brier_score(et_train, et_test, out_survival, times)[1])
roc_auc = []
for i, _ in enumerate(times):
    roc_auc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:, i], times[i])[0])
for horizon in enumerate(horizons):
    print(f"For {horizon[1]} quantile,")
    print("TD Concordance Index:", cis[horizon[0]])
    print("Brier Score:", brs[0][horizon[0]])
    print("ROC AUC ", roc_auc[horizon[0]][0], "\n")

For 0.25 quantile,
TD Concordance Index: 0.2456293458878584
Brier Score: 0.10939352729231262
ROC AUC  0.23506700811344128 

For 0.5 quantile,
TD Concordance Index: 0.3004553118898771
Brier Score: 0.183120045616337
ROC AUC  0.2791956689868523 

For 0.75 quantile,
TD Concordance Index: 0.33226686629755275
Brier Score: 0.21822007027782503
ROC AUC  0.28172290543405826 

